In this practice project, you will use the skills acquired through the course and create a complete ETL pipeline for accessing data from a website and processing it to meet the requirements.

## Project Scenario

An international firm that is looking to expand its business in different countries across the world has recruited you. You have been hired as a junior Data Engineer and are tasked with creating an automated script that can extract the list of all countries in order of their GDPs in billion USDs (rounded to 2 decimal places), as logged by the International Monetary Fund (IMF). Since IMF releases this evaluation twice a year, this code will be used by the organization to extract the information as it is updated.

You can find the required data on this [webpage](https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29).

The required information needs to be made accessible as a JSON file 'Countries_by_GDP.json' as well as a table 'Countries_by_GDP' in a database file 'World_Economies.db' with attributes 'Country' and 'GDP_USD_billion.'

Your boss wants you to demonstrate the success of this code by running a query on the database table to display only the entries with more than a 100 billion USD economy. Also, log the entire process of execution in a file named 'etl_project_log.txt'.

You must create a Python code 'etl_project_gdp.py' that performs all the required tasks.

## Objectives

You have to complete the following tasks for this project

1. Write a data extraction function to retrieve the relevant information from the required URL.

2. Transform the available GDP information into 'Billion USD' from 'Million USD'.

3. Load the transformed information to the required CSV file and as a database file.

4. Run the required query on the database.

5. Log the progress of the code with appropriate timestamps.

In [30]:
import pandas as pd
import sqlite3
import requests
from bs4 import BeautifulSoup
import myfunc as fun

### Scrape data

In [2]:
url = "https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29"

html_page = requests.get(url).text
data = BeautifulSoup(html_page, 'html.parser')

tables = data.find_all("tbody")
tables[2]

<tbody><tr class="static-row-header" style="text-align:center;vertical-align:bottom;">
<th rowspan="2">Country/Territory
</th>
<th rowspan="2"><a href="/web/20230902185326/https://en.wikipedia.org/wiki/United_Nations_geoscheme" title="United Nations geoscheme">UN region</a>
</th>
<th colspan="2"><a href="/web/20230902185326/https://en.wikipedia.org/wiki/International_Monetary_Fund" title="International Monetary Fund">IMF</a><sup class="reference" id="cite_ref-GDP_IMF_2-2"><a href="#cite_note-GDP_IMF-2">[1]</a></sup><sup class="reference" id="cite_ref-15"><a href="#cite_note-15">[13]</a></sup>
</th>
<th colspan="2"><a href="/web/20230902185326/https://en.wikipedia.org/wiki/World_Bank" title="World Bank">World Bank</a><sup class="reference" id="cite_ref-16"><a href="#cite_note-16">[14]</a></sup>
</th>
<th colspan="2"><a href="/web/20230902185326/https://en.wikipedia.org/wiki/United_Nations" title="United Nations">United Nations</a><sup class="reference" id="cite_ref-UN_17-0"><a href="#ci

In [3]:
rows = tables[2].find_all("tr")

df = pd.DataFrame(columns=["Country/Territory","UN_Region","IMF_Estimate", "IMF_Year"])

for row in rows:
    col = row.find_all("td")
    if len(col) != 0:
        data_dict = {"Country/Territory": col[0].text,
                     "UN_Region": col[1].contents[0],
                     "IMF_Estimate": col[2].contents[0],
                     "IMF_Year": col[3].text}
        df1 = pd.DataFrame(data_dict, index=[0])
        df = pd.concat([df,df1], ignore_index=True)

df

,Country/Territory,UN_Region,IMF_Estimate,IMF_Year
0,World,—,"105,568,776",2023
1,United States,Americas,"26,854,599",2023
2,China,Asia,"19,373,586",[n 1]2023
3,Japan,Asia,"4,409,738",2023
4,Germany,Europe,"4,308,854",2023
...,...,...,...,...
209,Anguilla,Americas,—,—
210,Kiribati,Oceania,248,2023
211,Nauru,Oceania,151,2023
212,Montserrat,Americas,—,—


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Country/Territory  214 non-null    object
 1   UN_Region          214 non-null    object
 2   IMF_Estimate       214 non-null    object
 3   IMF_Year           214 non-null    object
dtypes: object(4)
memory usage: 6.8+ KB


### Convert data in "IMF Estimate" and "IMF Year" columns into integer

In [5]:
df["IMF_Estimate"] = df["IMF_Estimate"].transform(lambda x: x.replace(",", ""))
df["IMF_Estimate"] = df["IMF_Estimate"].transform(lambda x: x.replace("—", 0))
df["IMF_Estimate"] = df["IMF_Estimate"].astype(int)

In [6]:
df["IMF_Estimate"].unique()

array([105568776,  26854599,  19373586,   4409738,   4308854,   3736882,
         3158938,   2923489,   2169745,   2089672,   2081235,   2062649,
         1721909,   1707548,   1663164,   1492432,   1391778,   1080880,
         1061902,   1029303,    869601,    790728,    748887,    641102,
          624248,    599052,    594095,    574231,    554105,    539223,
          515548,    515199,    506601,    498978,    449094,    447026,
          440901,    420516,    405626,    399015,    382854,    378110,
               0,    367970,    358557,    348902,    334689,    330483,
          301670,    268235,    267893,    267721,    251969,    245695,
          239300,    219570,    206007,    188505,    164713,    156083,
          148712,    138781,    127533,    121291,    121289,    120838,
          118130,    117877,    104902,    102309,    100635,     96628,
           92332,     86971,     85421,     82649,     78881,     78346,
           77777,     77313,     77257,     77047, 

In [7]:
df["IMF_Year"].unique()

array(['2023', '[n 1]2023', '[n 4]2023', '[n 5]2023', '376,533',
       '[n 6]2023', '107,352', '74,404', '[n 9]2023', '[n 10]2023',
       '[n 11]2023', '—', '[n 12]2023', '[n 14]2023', '7,551', '1,139',
       '1,572'], dtype=object)

In [8]:
df[df["IMF_Year"]=="107,352"]

,Country/Territory,UN_Region,IMF_Estimate,IMF_Year
68,Cuba,Americas,0,"107,352"


In [9]:
def update_year(row):
    if len(row) == 4:
        row = int(row)
        return row
    elif row.find("]") != -1:
        row = row.split("]")[1]
        row = int(row)
        return row
    else:
        row = 0
        return row

In [10]:
df["IMF_Year"] = df["IMF_Year"].apply(update_year)

In [11]:
df["IMF_Year"].unique()

array([2023,    0], dtype=int64)

In [12]:
df

,Country/Territory,UN_Region,IMF_Estimate,IMF_Year
0,World,—,105568776,2023
1,United States,Americas,26854599,2023
2,China,Asia,19373586,2023
3,Japan,Asia,4409738,2023
4,Germany,Europe,4308854,2023
...,...,...,...,...
209,Anguilla,Americas,0,0
210,Kiribati,Oceania,248,2023
211,Nauru,Oceania,151,2023
212,Montserrat,Americas,0,0


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Country/Territory  214 non-null    object
 1   UN_Region          214 non-null    object
 2   IMF_Estimate       214 non-null    int32 
 3   IMF_Year           214 non-null    int64 
dtypes: int32(1), int64(1), object(2)
memory usage: 6.0+ KB


### The data in the "IMF Estimate" column is in USD Million. Let's change it into USD Billion and round it up to 2 decimal places

In [14]:
df = df.rename({"IMF_Estimate": "IMF_Estimate_USD_Mio"}, axis=1)

In [15]:
df

,Country/Territory,UN_Region,IMF_Estimate_USD_Mio,IMF_Year
0,World,—,105568776,2023
1,United States,Americas,26854599,2023
2,China,Asia,19373586,2023
3,Japan,Asia,4409738,2023
4,Germany,Europe,4308854,2023
...,...,...,...,...
209,Anguilla,Americas,0,0
210,Kiribati,Oceania,248,2023
211,Nauru,Oceania,151,2023
212,Montserrat,Americas,0,0


In [16]:
df["IMF_Estimate_USD_Bio"] = df["IMF_Estimate_USD_Mio"].transform(lambda x: x/1000)
df["IMF_Estimate_USD_Bio"] = df["IMF_Estimate_USD_Bio"].transform(lambda x: round(x, 2))
df

,Country/Territory,UN_Region,IMF_Estimate_USD_Mio,IMF_Year,IMF_Estimate_USD_Bio
0,World,—,105568776,2023,105568.78
1,United States,Americas,26854599,2023,26854.60
2,China,Asia,19373586,2023,19373.59
3,Japan,Asia,4409738,2023,4409.74
4,Germany,Europe,4308854,2023,4308.85
...,...,...,...,...,...
209,Anguilla,Americas,0,0,0.00
210,Kiribati,Oceania,248,2023,0.25
211,Nauru,Oceania,151,2023,0.15
212,Montserrat,Americas,0,0,0.00


### Save data in JSON and DB

In [27]:
filename = 'Countries_by_GDP.csv'
table_name = 'Countries_by_GDP'
db_name = 'World_Economies.db'
attributes = ['Country', 'GDP_USD_billion']
logfile = 'etl_project_log.txt'

In [18]:
final_df = df.copy()
final_df = final_df.rename({"Country/Territory": "Country",
                            "IMF_Estimate_USD_Bio": "GDP_USD_billion"}, axis=1)
final_df = final_df[attributes]
final_df

,Country,GDP_USD_billion
0,World,105568.78
1,United States,26854.60
2,China,19373.59
3,Japan,4409.74
4,Germany,4308.85
...,...,...
209,Anguilla,0.00
210,Kiribati,0.25
211,Nauru,0.15
212,Montserrat,0.00


In [19]:
final_df.to_csv(filename)

In [20]:
conn = sqlite3.connect(db_name)
final_df.to_sql(name=table_name, con=conn, if_exists='replace', index=False)
conn.close()

### Let's check the DB and log the execution process

In [25]:
conn = sqlite3.connect(db_name)
query = pd.read_sql(f"SELECT * FROM {table_name} WHERE GDP_USD_billion > 100", conn)
query

,Country,GDP_USD_billion
0,World,105568.78
1,United States,26854.60
2,China,19373.59
3,Japan,4409.74
4,Germany,4308.85
...,...,...
65,Kenya,118.13
66,Angola,117.88
67,Oman,104.90
68,Guatemala,102.31


In [26]:
conn.close()

In [31]:
message = "Query has been run succesfully"

fun.execution_log(message=message, logfile=logfile)